In [ ]:
# Import needed packages
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [ ]:
## import the "raw" data
bbgaPD = pd.read_excel("C:\\VERTROUWELIJK\\2020_BBGA_0907.xlsx", engine='openpyxl')
youthCareData = pd.read_excel("C:\\VERTROUWELIJK\\20200902-Studentbegeleiding-JHoenderdos-Input-Hashed.xlsx")

In [ ]:
# Place to variables
jarenVanData = [2018,2019]

In [ ]:
# Place to make functions
def calculateAge(born):
    today = datetime.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [ ]:
## Describe the raw YouthCare data
youthCareData2018 = youthCareData[(youthCareData['Betrekking Op Jaar'] == 2018)]
youthCareData2018['Datum Geboorte'] = pd.to_datetime(youthCareData2018['Datum Geboorte'], format="%Y-%m-%d")
youthCareData2018['Leeftijd'] = youthCareData2018['Datum Geboorte'].apply(lambda x: calculateAge(x))

youthCareData2019 = youthCareData[(youthCareData['Betrekking Op Jaar'] == 2019)]
youthCareData2019['Datum Geboorte'] = pd.to_datetime(youthCareData2019['Datum Geboorte'], format="%Y-%m-%d")
youthCareData2019['Leeftijd'] = youthCareData2019['Datum Geboorte'].apply(lambda x: calculateAge(x))

number_of_rows_2018 = len(youthCareData2018.index)
number_of_rows_2019 = len(youthCareData2019.index)
uniqueID2018 = len(youthCareData2018['hashID'].unique())
uniqueID2019 = len(youthCareData2019['hashID'].unique())

averageAge2018 = youthCareData2018['Leeftijd'].mean()
averageAge2019 = youthCareData2019['Leeftijd'].mean()

sex2018 = youthCareData2018['Geslacht'].value_counts()
sex2019 = youthCareData2019['Geslacht'].value_counts()


In [ ]:
# BBGA data
bbgaPD = bbgaPD[(bbgaPD["niveaunaam"] == "4-cijferige postcode") & (bbgaPD.jaar.isin(jarenVanData))]
bbgaPD = bbgaPD.dropna(axis=1, how='all')

# for merging purposes, we change the name of one column
bbgaPD =bbgaPD.rename(columns = {'gebiedcode15':'Postcode'})
bbgaPD['Postcode'] = bbgaPD['Postcode'].astype(str).astype(int)

# removing colums, were there was to much NA values. 
bbgaPD = bbgaPD.drop([ 'niveau' , 'niveaunaam' , 'gebiedcodenaam','gebiednaam'], axis=1)


In [ ]:
# YouthCare Data
# Filter to get only data beloning to the right year
youthCareData = youthCareData[(youthCareData['Betrekking Op Jaar'].isin(jarenVanData))]

# remove all non digits in order to get Postcode4
youthCareData = youthCareData[youthCareData.Postcode != 'xxxxx']
youthCareData['Postcode'] = youthCareData['Postcode'].str.extract('(\d+)', expand=False)


# add extra colum to caluclate age in order te group persons better
youthCareData['Datum Geboorte'] = pd.to_datetime(youthCareData['Datum Geboorte'], format="%Y-%m-%d")
youthCareData['Leeftijd'] = youthCareData['Datum Geboorte'].apply(lambda x: calculateAge(x))
youthCareData['Postcode'] = youthCareData['Postcode'].astype(str).astype(int)


# Add regex in order to filter only real voorziening data
pat = '^[B|C][0-9][A-Z]{1,2}$'
youthCareData['VoorziningenRegex'] = youthCareData.Voorziening.str.contains(pat)
youthCareData = youthCareData[youthCareData['VoorziningenRegex'] == True]
youthCareData['Geslacht'] =youthCareData['Geslacht'].replace(['M', 'V'], [0,1])

In [ ]:
# merging data
merged_dataRegex = pd.merge(youthCareData, bbgaPD, on='Postcode')

In [ ]:
# create the final data set with 95% of the upper samples
Amount_of_Samples_delete = len(merged_dataRegex)-round(len(merged_dataRegex)*0.95)
overviewRegexAscendig = merged_dataRegex['Voorziening'].value_counts( ascending=True)
samplesToBeDeleted = pd.Series([])

sum = 0;
i = 0;
index = 0;

for i in overviewRegexAscendig:
    index = index + 1;
    sum = i + sum;
    
    if(sum < Amount_of_Samples_delete):
        continue;
    else:
        samplesToBeDeleted = overviewRegexAscendig[0:index];
        break;
        
tempdf = samplesToBeDeleted.to_frame()

tempdf = tempdf.reset_index()
serviceNamesToBeDeleted = tempdf['index'].tolist()
finalDataSet = merged_dataRegex[~merged_dataRegex['Voorziening'].isin(serviceNamesToBeDeleted)]
finalDataSet = finalDataSet.reset_index(drop=True)

# removing last variables which does not help in making the model
finalDataSet = finalDataSet.drop(['hashID', 'Datum Geboorte', 'Productcategorie', 'Voorzieningsoort', 'Productcategorie', 'Betrekking Op Jaar', 'VoorziningenRegex', 'jaar' ], axis=1)

finalDataSet = finalDataSet.fillna(0);

In [ ]:
## make an plot for the report
overviewRegex = merged_dataRegex['Voorziening'].value_counts()
fig = plt.figure()
plot = overviewRegex.plot.line()
plot.axhline(y=132, color='r', linestyle='-')
plot.set_xlabel('Services', fontsize=14)
plot.set_ylabel('Frequency', fontsize=14)
fig.add_subplot(plot)
fig.savefig('full_figure.png')

In [ ]:
finalDataSet.to_pickle("C:\\VERTROUWELIJK\\final_dataSet.pkl")